In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from grid import generate_lats_lons
from paths import path_results, path_samoc, file_ex_ice_rcp, file_ex_ice_yrly, file_ex_ocn_rcp
from plotting import discrete_cmap, view_colormap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_rcp, decode_times=False).REGION_MASK

In [ ]:
# %%time
# # stacking files into one xr Dataset object
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ice', run, 'monthly')):
#         if i==0:
#             print(i,m,y,s)
#             start_year = y
#             tarea = xr.open_dataset(s, decode_times=False).tarea[:,:]
            
#         if m==3:  # March Arctic max, Antarctic min
#             da = xr.open_dataset(s, decode_times=False).aice[0,:,:]
#             if y==start_year:
#                 da1_new = da.copy()
#             else:
#                 da1_new = xr.concat([da1_new, da], dim='time')
                
#         elif m==9:  # September Arctic min, Antartis max
#             da = xr.open_dataset(s, decode_times=False).aice[0,:,:]
#             if y==start_year:
#                 da2_new = da.copy()
#             else:
#                 da2_new = xr.concat([da2_new, da], dim='time')

#     da1_new = xr.merge([da1_new, tarea]) 
#     da2_new = xr.merge([da2_new, tarea]) 
                
#     da1_new.to_netcdf(f'{path_samoc}/CICE/CICE_Mar_{run}.nc')
#     da2_new.to_netcdf(f'{path_samoc}/CICE/CICE_Sep_{run}.nc')
    
# # 8 min

In [ ]:
ctrl_M = xr.open_dataset(f'{path_samoc}/CICE/CICE_Mar_ctrl_.nc', decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/CICE/CICE_Sep_ctrl_.nc', decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/CICE/CICE_Mar_rcp.nc' , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/CICE/CICE_Sep_rcp.nc' , decode_times=False)

In [ ]:
# %%time
# # ca. 3:30 min
# ctrl_Arctic_max    = (ctrl_M.tarea*ctrl_M.aice).where(ctrl_M.nj>1200).sum(dim=('ni', 'nj'))
# ctrl_Arctic_min    = (ctrl_S.tarea*ctrl_S.aice).where(ctrl_S.nj>1200).sum(dim=('ni', 'nj'))
# ctrl_Antarctic_min = (ctrl_M.tarea*ctrl_M.aice).where(ctrl_M.nj<1200).sum(dim=('ni', 'nj'))
# ctrl_Antarctic_max = (ctrl_S.tarea*ctrl_S.aice).where(ctrl_S.nj<1200).sum(dim=('ni', 'nj'))

# rcp_Arctic_max     = (rcp_M.tarea *rcp_M.aice ).where(rcp_M.nj >1200).sum(dim=('ni', 'nj'))
# rcp_Arctic_min     = (rcp_S.tarea *rcp_S.aice ).where(rcp_S.nj >1200).sum(dim=('ni', 'nj'))
# rcp_Antarctic_min  = (rcp_M.tarea *rcp_M.aice ).where(rcp_M.nj <1200).sum(dim=('ni', 'nj'))
# rcp_Antarctic_max  = (rcp_S.tarea *rcp_S.aice ).where(rcp_S.nj <1200).sum(dim=('ni', 'nj'))

# ctrl_aice = xr.merge([{'Arctic_max'   : ctrl_Arctic_max, 
#                        'Arctic_min'   : ctrl_Arctic_min,
#                        'Antarctic_max': ctrl_Antarctic_max, 
#                        'Antarctic_min': ctrl_Antarctic_min }])
# rcp_aice  = xr.merge([{'Arctic_max'   : rcp_Arctic_max, 
#                        'Arctic_min'   : rcp_Arctic_min,
#                        'Antarctic_max': rcp_Antarctic_max, 
#                        'Antarctic_min': rcp_Antarctic_min }])

# ctrl_aice.to_netcdf(f'{path_results}/CICE/CICE_timeseries_ctrl.nc')
# rcp_aice.to_netcdf( f'{path_results}/CICE/CICE_timeseries_rcp.nc' )

In [ ]:
ctrl_aice = xr.open_dataset(f'{path_results}/CICE/CICE_timeseries_ctrl.nc', decode_times=False)
rcp_aice  = xr.open_dataset(f'{path_results}/CICE/CICE_timeseries_rcp.nc' , decode_times=False)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(16,5))

ax[0].plot(ctrl_aice.time[0::2]/365    , ctrl_aice.Arctic_max.dropna(dim='time')/1e14   , c='C0', lw=2, ls='--', label='Arctic CTRL')
ax[0].plot(rcp_aice.time[0::2]/365-1800, rcp_aice.Arctic_max.dropna(dim='time') /1e14   , c='C1', lw=2, ls='--', label='Arctic RCP')
ax[0].plot(ctrl_aice.time[0::2]/365    , ctrl_aice.Antarctic_max.dropna(dim='time')/1e14, c='C0', lw=2, ls='-' , label='Antarctic CTRL')
ax[0].plot(rcp_aice.time[0::2]/365-1800, rcp_aice.Antarctic_max.dropna(dim='time') /1e14, c='C1', lw=2, ls='-' , label='Antarctic RCP')
ax[0].text(.02,.93, 'max extent', ha='left', transform=ax[0].transAxes, fontsize=16)

ax[1].axhline(0, c='k', lw=.5)
ax[1].plot(ctrl_aice.time[0::2]/365    , ctrl_aice.Arctic_min.dropna(dim='time')/1e14   , c='C0', lw=2, ls='--', label='Arctic CTRL')
ax[1].plot(rcp_aice.time[0::2]/365-1800, rcp_aice.Arctic_min.dropna(dim='time') /1e14   , c='C1', lw=2, ls='--', label='Arctic RCP')
ax[1].plot(ctrl_aice.time[0::2]/365    , ctrl_aice.Antarctic_min.dropna(dim='time')/1e14, c='C0', lw=2, ls='-' , label='Antarctic CTRL')
ax[1].plot(rcp_aice.time[0::2]/365-1800, rcp_aice.Antarctic_min.dropna(dim='time') /1e14, c='C1', lw=2, ls='-' , label='Antarctic RCP')
ax[1].text(.02,.93, 'min extent', ha='left', transform=ax[1].transAxes, fontsize=16)
ax[1].set_ylim((-.1,3.1))

for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_xlabel('time [years]', fontsize=16)
    ax[i].set_ylabel('Sea Ice area [10$^6$ km$^2$]', fontsize=16)
    ax[i].legend(fontsize=16, ncol=2, frameon=False)

plt.tight_layout()
plt.savefig(f'{path_results}/CICE/CICE_areas_timeseries')

# Polynya area
CESM polynya region of Rene: 63.5-66.5 S x 2-11 E

In [ ]:
pa = rcp_S.tarea.where(rcp_S.tarea<1e11)  # polynya sea ice area
pa = pa.where(pa.TLAT>-66.5)
pa = pa.where(pa.TLAT<-63.5)
pa = pa.where(pa.TLON>3    )
pa = pa.where(pa.TLON<11   )
pa_full = pa.sum(dim=('ni','nj'))

In [ ]:
pa.plot()

In [ ]:
ctrl_polynya_aice = (pa*rcp_S.aice ).sum(dim=('ni','nj'))
rcp_polynya_aice  = (pa*ctrl_S.aice).sum(dim=('ni','nj'))

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.plot(ctrl_polynya_aice/pa_full, lw=2, label='CTRL')
plt.plot(rcp_polynya_aice /pa_full, lw=2, label='RCP')
plt.ylim((0,100))

plt.text(.02,.95, r'[63.5,66.5$^\circ$S]$\times$[3,11$^\circ$E] during max extent', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('Sea Ice cover [%]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
plt.savefig(f'{path_results}/CICE/CICE_cover_polynya_region')

# Sea ice MXL plot

In [ ]:
aice = xr.open_dataset(file_ex_ice_rcp, decode_times=False).aice[0,:,:]
XMXL = xr.open_dataset(file_ex_ocn_rcp, decode_times=False).XMXL[0,:,:]

In [ ]:
lats, lons = generate_lats_lons('ocn')

In [ ]:
ax = plt.subplot(1,1,1, projection=ccrs.Orthographic(central_longitude=0, central_latitude=90))
ax.pcolormesh(lons, lats, aice,
              cmap=cmocean.cm.ice, vmin=0, vmax=100,
              transform=ccrs.PlateCarree() )

In [ ]:
%%time
CICE_XMXL_plots(aice, XMXL, lons, lats, MASK, 'rcp', 0)

In [ ]:
view_colormap( cmocean.cm.balance)

In [ ]:
view_colormap(discrete_cmap(7, cmocean.cm.balance))